## Postprocess and Analyze

In [ ]:
import logging
logging.basicConfig(filename='example.log', encoding='utf-8', level=logging.INFO)

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import fracsim.measure.isolines as isolines_calculator
from fracsim.measure.curvature import calc_local_curvature, calc_local_curvature_old
import fracsim.simulation.plotting as plotting
import fracsim.simulation.analysis as analysis
import fracsim.simulation.postprocessing as postprocessing

In [ ]:
%%capture
test = False

do_postprocessing = False
if do_postprocessing:
    df = pd.read_csv("results/simulation_results.csv", index_col=0)
    if test:
        df = df.tail(5)
    simulation_results = df.to_dict(orient="records")
    postprocessing_results = postprocessing.postprocess(simulation_results)
    pd.DataFrame(postprocessing_results).to_csv("results/postprocessing_results.csv")

do_analysis = True
analysis_params = {
    "iso_value":0.98,
    "curvature_smoothing_stride": 3,#3 in paper!
    "gaussian_smoothing_sigma": 10.0,
}
if do_analysis:
    df = pd.read_csv("results/postprocessing_results.csv", index_col=0)
    if test:
        df = df.tail(5)
    postprocessing_results = df.to_dict(orient="records")
    analysis_results = analysis.analyze(postprocessing_results, **analysis_params)
    df = pd.DataFrame(analysis_results)
    df.to_csv("results/analysis_results.csv")

## Plot

In [ ]:
# remove runs with skeleton_length smaller than 500, they are faulty
df = pd.read_csv("results/analysis_results.csv")
df = df[df["skeleton_length"] > 500]

# df = df[(df["std_lambda"] == 5.05e8/4) | (df["homogeneous"] == True)]
# is_in_wanted_lengthscale = df["lengthscale_lambda"]== 1
# is_in_wanted_lengthscale = np.logical_or(is_in_wanted_lengthscale, df["lengthscale_lambda"]== 5)
# is_in_wanted_lengthscale = np.logical_or(is_in_wanted_lengthscale, df["lengthscale_lambda"]== 1000)

plots = ["volume", "fractal_dimension", #"count"
         "isoline_length", "skeleton_length",
         #"total_curvature",
         "skeleton_y_max",
        #  "mean_curvature_smoothed",
        #  "max_curvature", "mean_curvature",
         "sign_changes", "sign_changes_smoothed",
        # "isoline_y_max",
    ]# "isoline_n_points", "isoline_mean_segment_length"]

df_sub = df#[is_in_wanted_lengthscale]
plotting.plot(df_sub, create_subplots=True, plots=plots, show_mean=False)#, show_run_idx=True)

In [ ]:
from scipy.ndimage import gaussian_filter1d

# random_indices = np.random.randint(0, len(df), 10)
all_indices = np.arange(len(df))
random_indices = all_indices

y_min, y_max = -150, 150 #df["structured_mesh_min_y"].min(), df["structured_mesh_max_y"].max()

for i in random_indices:
    # Select row by using the column "run" which contains the run number which has the same range as the index
    df_row = df.iloc[i]
    OP_01 = np.load(df_row["OP_01"])
    isolines = isolines_calculator.isoline(OP_01, iso_value=df_row["iso_value"])
    # plt.figure()
    # curvatures = []
    # for isoline in isolines:
    #     isoline_curvature = curvature.calc_curvature(isoline, stride=20)
    #     curvatures.append(isoline_curvature)

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax2.set_ylabel('Curvature', color='b')
    ax1.set_ylabel('y')
    ax1.set_ylim(y_min, y_max)
    stride = 3
    # ax1.set_ylim(y_min, y_max)
    curvature_sum_sum = 0
    curve_int_sum = 0
    curvature_mean = 0
    curvature_max = -1
    n_sign_changes = 0
    for isoline in isolines[0:2]:
        # if np.max(isoline[:, 0]) - np.min(isoline[:, 0]) < 100:
        #     continue
        isoline_curvature_local = calc_local_curvature(isoline, stride=stride)
        smoothed_curve = gaussian_filter1d(isoline_curvature_local, 5.0)
        ax2.plot(isoline[::stride,1], smoothed_curve, color='g')
        curve_int = np.trapz(np.abs(isoline_curvature_local), np.sqrt(isoline[::stride,1]**2+isoline[::stride,0]**2))
        curve_int_sum += curve_int
        curvature_sum = np.sum(np.abs(isoline_curvature_local))
        curvature_sum_sum += curvature_sum
        curvature_max = max(np.max(np.abs(isoline_curvature_local)), curvature_max)
        # x = np.linspace(0, 1000, len(isoline_curvature_local))
        # ax2.plot(isoline[::stride,1], isoline_curvature_local, color='b')
        ax1.plot(isoline[::stride,1], isoline[::stride,0]-250, color='r')
        print(len(isoline))
        print("Plot with index ", i)
        print(f"max curvat: {curvature_max}")
        n_sign_changes += np.sum(np.diff(np.sign(smoothed_curve)) != 0)

        # # A histrogram of the absolute curvature values
        # plt.figure()
        # plt.hist(np.abs(isoline_curvature_local), bins=100)
        # # get the lengthscale_lambda from the df_row
        # lengthscale_lambda = df_row["lengthscale_lambda"]
        # # vertical line at 1/lengthscale_lambda
        # plt.axvline(1/lengthscale_lambda, color='r')
        # # the mean of the absolute curvature values
        # curvature_mean = np.mean(np.abs(isoline_curvature_local))
        # estimate_lengthscale_lambda = 1/curvature_mean
        # plt.title(f"lengthscale_lambda={lengthscale_lambda}, estimate={estimate_lengthscale_lambda}")
        # plt.axvline(curvature_mean, color='g')
        # plt.axvline(np.median(np.abs(isoline_curvature_local)), color='b')
        # plt.show()
    # ax1.set_title(f"curvature_sum={curvature_sum_sum}") #, curvature={isoline_curvature:.2f}")
    print(f"n_sign_changes: {n_sign_changes}")
    plt.show()